Prerequisite packages
* import error: 環境沒有的套件google一下就會有教學怎麼裝

In [ ]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *
from keras.models import model_from_json, model_from_yaml
from sklearn.metrics import f1_score,recall_score,precision_score,hamming_loss , roc_auc_score

# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

在與此code相同的目錄裡，要包含本github( https://github.com/tychen5/NTU_ANTS-lab/tree/master/seqgen_famclf )中的相關額外檔案，才可以順利import

In [ ]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

Read & Restore data prepare by 0003.

In [ ]:
# train/dev/test
root_dir = 'data/tree-rep-profiles_o2o/' #0003存放各個dataset的pkl目錄
train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open(root_dir + 'pids_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open(root_dir + 'pids_test.pkl','rb'))
exp_api,exp_emb = pickle.load(open(root_dir + 'pids_exp.pkl','rb'))
# print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
# print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
# train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train_only.pkl','rb'))
# train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
valid_rep_ans = np.expand_dims(valid_rep_ans,axis=-1)
test_rep_ans = np.expand_dims(test_rep_ans,axis=-1)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_rep_ans.shape)
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb')) #embedding layer initialize weights

In [ ]:
def _shuffle(X, X2 ,X3):
    '''
    將train set資料順序打亂，不然NN會train壞
    '''
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
    return (X[randomize], X2[randomize],X3[randomize])

In [ ]:
train_emb, train_emb_api, train_rep_ans = _shuffle(train_emb, train_emb_api, train_rep_ans)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_rep_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_rep_ans.shape)

# NN Model

### Hyper-parameters

In [ ]:
emb_dim = valid_emb.shape[-1] #被除數
num_heads = 8 #Transformer multi-head實驗才要用
max_length = test_emb_api.shape[1] # max sequence length
# fam_num = train_fam_ans.shape[1]
vocabulary_size = emb_matrix.shape[0]-1 #API function name種類
transformer_depth = 1 # transformer實驗 block數目
transformer_dropout = 0.1 #transformer 實驗模型預設do
l2_reg_penalty = 1e-6#1e-4
# dp_rate = 0.01 #dropout，沒用到

# traina = True #沒用到
batch_size = 128 
learning_rate = 0.001#5e-4#2e-4 #初始learning rate
num_epochs = 1000
patien = 30 # early stopping條件
model_save_path = './model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5' #模型儲存位置
tensorboard_log_path = './logs/'+ model_save_path.split('/')[-1].split('.')[0] #history log path

transformer model 實驗才會用到
* transformer encoder: https://github.com/kpot/keras-transformer

In [ ]:
# constrain = keras.constraints.MinMaxNorm(min_value=0.0, max_value=0.0, rate=1.0, axis=0)
# init = keras.initializers.Ones() #
coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth , name='coordinate_embedding') #transformer的 positional encoding
act_layer = TransformerACT(
            name='adaptive_computation_time')

transformer_block = TransformerBlock(
            name='transformer', num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            # Allow bi-directional attention
            use_masking=False)
add_segment_layer = Add(name='add_segment')
l2_regularizer = (regularizers.l2(l2_reg_penalty) if l2_reg_penalty else None)

### Architecture building

input

In [ ]:
sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb') #輸入sent2vec vectors
sentemb = Masking(mask_value=0)(sentemb1) #0 is padding

sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1) #API整數編碼。0 is padding
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length
                    ,trainable=True,name='api_emb',trainable=True)(sent_ids) #embedding layer

segment_embeddings = Add()([sentemb,api_emb]) #相加兩種embedding vectors

GRU & self-attention

In [ ]:
att_rnn = GRU(int(emb_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(segment_embeddings) #RNN
att_rnn = BatchNormalization(name='bn')(att_rnn) #batch feature scaling
self_att = SeqSelfAttention(attention_activation='sigmoid',name='self_attention')(att_rnn) #github預設的self-attention mechanism: https://pypi.org/project/keras-self-attention/


output

In [ ]:
rep_prediction = (
        TimeDistributed(Dense(1, name='0_1_predict', activation='sigmoid',trainable=True),name='out_rep') # hard_sigmoid
    (self_att)) #prediction layer

In [ ]:
model = Model(inputs=[sent_ids1,sentemb1], outputs=[rep_prediction]) #build model graph
model.summary()

## Loss/Optimizer/Metrics

available metrics

In [ ]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


def custom_acc1(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred,k=3)
from keras.metrics import binary_accuracy
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

def precision(y_true, y_pred):
    return tf.keras.metrics.Precision(y_true,y_pred)[1]
def recall(y_true, y_pred):
    return tf.keras.metrics.Recall(y_true,y_pred)[1]


metric = [bin_acc]

available loss

In [ ]:
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))
def hn_multilabel_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
from keras.metrics import top_k_categorical_accuracy

los = [losses.binary_crossentropy]
loss_weight = [1] #如果是multi-task learning才要改

model initialize

In [ ]:
optimizer = optimizers.Adam(
            lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False, clipvalue=1.) #clipnorm=1. , clipvalue=1.防止gradient explosion
# optimizer = keras.optimizers.Nadam(lr=learning_rate, clipvalue=1.) #另外一種adam optomizer

lr_scheduler1 = callbacks.LearningRateScheduler( # lr隨著epoch遞減
        CosineLRSchedule(lr_high=learning_rate, lr_low=1e-8, #learning_rate 
                         initial_period=num_epochs),
        verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/3),
                                      min_lr=1e-8,mode='min') #當model的loss卡住的時候(連續三分之一個patien epoch沒進步)會立刻把lr砍半

model.compile(
            optimizer,
            loss=los,
            metrics=metric ,loss_weights=loss_weight)

save best model only, early stopping

In [ ]:
history = History()
stop_nan = callbacks.TerminateOnNaN() # train 壞了立刻中止
model_callbacks = [
        callbacks.ModelCheckpoint( #只保留model在DEV表現最好的那個epoch之模型參數
            model_save_path, 
            monitor='val_loss',mode='min' ,save_best_only=True, verbose=1,save_weights_only=True), 
            EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'), #依據valid loss決定是否終止訓練
        lr_scheduler,history,stop_nan ,lr_scheduler1
    ]
model_callbacks.append(callbacks.TensorBoard(tensorboard_log_path))

準備每一個batch的data

In [ ]:
def training_batch(batch_size, X_train1, X_train2 , Y_train1):#, Y_train2):
    '''
    X_train1 = sent_ids: shape為(N, max_seq_length)=>API function name整數編碼
    X_train2 = sentemb: shape為(N,max_seq_length, word_embedding_size)
    Y_train1 = binary vector prediction: shape為(N, max_seq_length, 1)
    '''
    idx = np.arange(len(X_train1))
    np.random.shuffle(idx) # batch內隨機打亂

    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
            yield ([train_X1,train_X2],[train_Y1]) #改
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break


### Train

In [ ]:

H = model.fit_generator( #train_rep_ans
    generator=training_batch(batch_size=batch_size,X_train1=train_emb_api,X_train2=train_emb ,
                                             Y_train1=train_rep_ans)
                        , steps_per_epoch=int(np.ceil(len(train_emb_api)/batch_size)) ,
                    epochs=num_epochs,callbacks=model_callbacks
                   ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]) 
                    ,max_queue_size=10 
                    ,workers=10,use_multiprocessing=True   
                   ,shuffle=True,verbose=1)
model.save(model_save_path+"_all.h5")


# Filter功能模組
* 挑選模型於驗證資料集上的最佳threshold

load model

In [ ]:
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)
# model_save_path =  './model/o2o_embEXP_gru_selfatt/sent2vec_mask_0718.h5' #實驗的model
model = load_model(model_save_path+'_all.h5',custom_objects={'bin_acc': bin_acc,'RemoveMask':RemoveMask,'RestoreMask':RestoreMask, 'SeqSelfAttention': SeqSelfAttention})
model.load_weights(model_save_path)
model.summary()

predict on DEV/Test Set

In [ ]:
ans_t = model.predict([test_emb_api,test_emb])
ans_v = model.predict([valid_emb_api,valid_emb])
print(ans_t.shape)

roc auc score

In [ ]:
y_true = test_rep_ans.squeeze(axis=-1)
ans_t = ans_t.squeeze(axis=-1)
area_score_t = roc_auc_score(y_true,ans_t,average='micro')
y_true = valid_rep_ans.squeeze(axis=-1)
ans_v = ans_v.squeeze(axis=-1)
area_score_v = roc_auc_score(y_true,ans_v,average='micro')
print('valid roc auc score:',area_score_v,'test:',area_score_t)

choose final threshold for the model
* 實驗1-1與1-2的表現
* 先選出模型在validation Set score最高(F1-hamming loss)的那個threshold
* 再去看那個threshold再testing Set上的F1、precision、recall、hamming_loss表現

In [ ]:
score_list = []
score_list_t = []
f1_list_v = []
pre_list_v = []
rec_list_v = []
f1_list_t = []
pre_list_t = []
rec_list_t = []
hloss_list_v = []
hloss_list_t = []
threashold_list = []
max_score = 0
for i in tqdm(range(1,100)): #0.01~0.99 #system: 500 (0.002)
    thr = i/100 # system: 500 (0.002)
    final_ans_t = []
    for sample in ans_t:
        sample_ans = []
        for value in sample:
            if value < thr: #0.26 #0.33
                sample_ans.append(0)
            else:
                sample_ans.append(1)
        final_ans_t.append(sample_ans)
    final_ans_t = np.array(final_ans_t)
#     print(final_ans_t.shape , sum(final_ans_t[0]))

    final_ans_v = []
    for sample in ans_v:
        sample_ans = []
        for value in sample:
            if value < thr: #0.26 #0.33
                sample_ans.append(0)
            else:
                sample_ans.append(1)
        final_ans_v.append(sample_ans)
    final_ans_v = np.array(final_ans_v)
#     print(final_ans_v.shape , sum(final_ans_v[0]))

    y_true = test_rep_ans.squeeze(axis=-1)
    recall_t = recall_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    precision_t = precision_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    f1_t = f1_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    h_loss_t = hamming_loss(y_true,final_ans_t)
#     print('Test: (recall/precision/f1/h_loss)',recall_t ,precision_t, f1_t , h_loss_t)

    y_true = valid_rep_ans.squeeze(axis=-1)
    recall_v = recall_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    precision_v = precision_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    f1_v = f1_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    h_loss_v = hamming_loss(y_true,final_ans_v)
    score = f1_v - h_loss_v
    score_t = f1_t - h_loss_t
    if score>max_score:
        print('Best choice threashold now is:',thr,
              '本模型最佳的validation F1,該thr對應的testing F1,最佳的validation hamming_loss,對應的testing hamming_loss:',
              f1_v,f1_t,h_loss_v,h_loss_t)
        max_score = score
    score_list.append(score)
    score_list_t.append(score_t)
    threashold_list.append(thr)
    f1_list_v.append(f1_v)
    pre_list_v.append(precision_v)
    rec_list_v.append(recall_v)
    f1_list_t.append(f1_t)
    pre_list_t.append(precision_t)
    rec_list_t.append(recall_t)
    hloss_list_v.append(h_loss_v)
    hloss_list_t.append(h_loss_t)
#     print('Valid: (recall/precision/f1/h_loss)=>',recall_v ,precision_v, f1_v , h_loss_v)
print('final thr(best in DEV set) is the last threshold that print out. 相減Score =',max_score)

### 不同Filter threshold實驗與匯出excel畫圖
* 實驗#1-3
* Sec. 4.2 Filter實驗

In [ ]:
score_df = pd.DataFrame(data={'threshold':threashold_list,'Precision_valid':pre_list_v,'Recall_valid':rec_list_v
                              ,'F1_valid':f1_list_v,'Hloss_valid':hloss_list_v
                              ,'score_valid':score_list,'Precision_test':pre_list_t,'Recall_test':rec_list_t,
                              'F1_test':f1_list_t,'Hloss_test':hloss_list_t
                              ,'score_test':score_list_t})
score_df.to_excel('data/tree-rep-profiles_o2o/threxp_F1_rec_pre_Hloss.xlsx',index=False) #輸出檔案位置
score_df

In [ ]:
# for EXP#1-1、EXP#1-2，觀察不同模型_test的表現 (要把threshold改成該模型再DEV set最佳的thr)
score_df[score_df['threshold']== 0.51]

## training process之loss與acc輸出至excel畫圖

利用history file輸出
* 記得更改H檔案存放路徑

In [ ]:
log_dir = './logs/MLP_2dense_0718/'
history = pickle.load(open(log_dir+'MLP_3dense_gruatt_H.pkl','rb')) #train過程所存放
keys = list(history.keys())
keys #有哪些指標可供提供

In [ ]:
history_df = pd.DataFrame(data={keys[0]:history[keys[0]], keys[1]:history[keys[1]],
                               keys[2]:history[keys[2]], keys[3]:history[keys[3]]})
history_df.to_excel('data/tree-rep-profiles_o2o/encEXP_MLP_history_0718.xlsx',index=True) #輸出志excel
history_df

利用TF events輸出
* 記得更改event file位置

In [ ]:
event_file = log_dir+'events.out.tfevents.1562415044.superpc4'
for event in tf.train.summary_iterator(event_file):
    for value in event.summary.value:
        print(value.tag) #有哪些指標可供提供
        break

In [ ]:
# e，即event，代表某一個batch的日誌記錄
for e in tf.train.summary_iterator(event_file):
    # v，即value，代表這個batch的某個已記錄的觀測值，loss或者accuracy
    for v in e.summary.value:
        if v.tag == 'loss' or v.tag == 'bin_acc': #輸出自己想要關注的指標
            print(v.tag,v.simple_value)

***

# Appendix Spplementary
* 比較不同方法實驗用的code (實驗一的MLP和Transformer)
* Need modifications

EXP#1-1: 更改NN architecture input的embedding layer或是Sent2Vec input就好了

EXP#1-2: 
* GRU只要architecture的地方留下GRU拿掉self-attention就好了
* Dense則只要把GRU跟self-attention改成以下，再輸出到output layer
```
dense1 = Dense(int(emb_dim/4),activation='relu',name='dense1')(segment_embeddings)
dense2 = Dense(int(emb_dim/4),activation='relu',name='dense2')(dense1)
```
* Transformer Encoder architecture 完整範例如以下

In [ ]:
sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)

api_emb = Embedding(vocabulary_size+1, emb_dim,input_length=max_length,weights=[emb_matrix]
                    ,trainable=True,name='api_emb')(sent_ids) #改


segment_embeddings = Add()([sentemb,api_emb])
removed_seg = RemoveMask()(segment_embeddings)
next_step_input = coordinate_embedding_layer(removed_seg, step=0)
next_step_input = add_segment_layer([next_step_input, removed_seg])
for i in range(transformer_depth): 
    next_step_input = (
        TransformerBlock(
            name='transformer' + str(i), num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            use_masking=False,  # Allow bi-directional attention
            vanilla_wiring=True)
        (next_step_input))
restored_seg = RestoreMask()([next_step_input,segment_embeddings])
att_rnn = BatchNormalization(name='bn')(restored_seg)

rep_prediction = (
        TimeDistributed(Dense(1, name='0_1_predict', activation='sigmoid',trainable=True),name='out_rep') # hard_sigmoid
    (att_rnn)) 

model = Model(inputs=[sent_ids1,sentemb1], outputs=[rep_prediction]) #out
model.summary()